# Assignment 6 - Telecom Customer NPS Analysis

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import pandas as pd
import numpy as np
import requests
import pickle
import json

## Load Dataset

In [53]:
k_telecom = pd.read_csv('/content/sample_data/k_telecom.csv', delimiter=';')
d_telecom = pd.read_csv('/content/sample_data/d_telecom.csv', delimiter=';')
abon_data = pd.read_csv('/content/sample_data/abon_profile.csv', delimiter=';')

In [54]:
k_telecom.head()

,SESSION_START_TIME,ID,SERVICE_ID,CELL_NUM,LAC_NUM,TARIFF_PLAN_NAME,PAY_AMT,SESSION_DURATION_IN_SEC,DIRECT_IND,TYPE_OF_CONNECTION,ID_CONNECTED_NUM
0,2019-03-03 11:26:07.0,62340,1412,CELL_35832,LAC_547,PRICE_PLAN_NAME_72,237.497400,324.0,2.0,voice,26919.0
1,2019-02-20 02:05:33.0,62637,1164,NaN,NaN,PRICE_PLAN_NAME_49,314.150000,NaN,NaN,NaN,NaN
2,2019-04-10 00:30:20.0,63002,1128,NaN,NaN,PRICE_PLAN_NAME_72,188.490000,NaN,NaN,NaN,NaN
3,2019-03-14 12:05:26.0,63163,1140,CELL_37450,LAC_853,PRICE_PLAN_NAME_64,69.270075,6543.0,NaN,gprs,NaN
4,2019-04-02 21:09:24.0,63978,1410,NaN,NaN,PRICE_PLAN_NAME_61,47.122500,0.0,2.0,sms,22557.0


In [55]:
d_telecom.head()

,SESSION_START_TIME,ID,SERVICE_ID,CELL_NUM,LAC_NUM,TARIFF_PLAN_NAME,PAY_AMT,SESSION_DURATION_IN_SEC,DIRECT_IND,TYPE_OF_CONNECTION,ID_CONNECTED_NUM
0,2019-05-19 11:27:51.0,73811,1140,CELL_2232,LAC_589,PRICE_PLAN_NAME_87,0.031415,0.0,NaN,gprs,NaN
1,2019-05-19 19:02:14.0,10707,1069,CELL_17104,LAC_646,PRICE_PLAN_NAME_105,0.125660,4451.0,NaN,gprs,NaN
2,2019-05-19 11:11:51.0,17602,1408,CELL_19102,LAC_548,PRICE_PLAN_NAME_88,1.916315,915.0,NaN,gprs,NaN
3,2019-05-19 20:59:47.0,10707,1052,CELL_17044,LAC_646,PRICE_PLAN_NAME_105,42.661570,576.0,NaN,gprs,NaN
4,2019-05-19 07:08:46.0,63626,1052,CELL_12316,LAC_545,PRICE_PLAN_NAME_87,0.157075,7256.0,NaN,gprs,NaN


In [56]:
abon_data.head()

,ID,VARIABLE_0,VARIABLE_1,VARIABLE_2,VARIABLE_3,VARIABLE_4,VARIABLE_5,VARIABLE_6,VARIABLE_7,VARIABLE_8,...,VARIABLE_60,VARIABLE_61,VARIABLE_62,VARIABLE_63,VARIABLE_64,VARIABLE_65,VARIABLE_66,VARIABLE_67,VARIABLE_68,VARIABLE_69
0,7991,NaN,NaN,NaN,NaN,0.00,0,0,0,31,...,1.000000,0.000000,1.000000,0.000000,-570.107000,148.727587,0.997222,0.011517,0.997222,0.011517
1,11480,16.0,7.0,2.0,3.0,8416.37,10,1,0,16,...,0.580833,0.359752,0.533500,0.348006,-300.683889,659.992067,0.698778,0.362723,0.666333,0.366837
2,12179,22.0,8.0,3.0,3.0,10780.00,9,1,1,0,...,0.225333,0.140537,0.875833,0.237439,249.989000,261.567349,0.220556,0.174087,0.705333,0.365302
3,13122,28.0,0.0,5.0,3.0,4140.00,8,2,2,3,...,0.000000,0.000000,0.909667,0.232517,223.085444,219.941389,0.000000,0.000000,0.816444,0.350855
4,15742,40.0,0.0,4.0,5.0,2985.73,3,1,0,3,...,0.284000,0.357511,0.232000,0.368822,631.502222,2180.477234,0.196667,0.317490,0.408444,0.460210


In [57]:
k_telecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76046 entries, 0 to 76045
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   SESSION_START_TIME       76046 non-null  object 
 1   ID                       76046 non-null  int64  
 2   SERVICE_ID               76046 non-null  int64  
 3   CELL_NUM                 26489 non-null  object 
 4   LAC_NUM                  26492 non-null  object 
 5   TARIFF_PLAN_NAME         76046 non-null  object 
 6   PAY_AMT                  76046 non-null  float64
 7   SESSION_DURATION_IN_SEC  37411 non-null  float64
 8   DIRECT_IND               23372 non-null  float64
 9   TYPE_OF_CONNECTION       37411 non-null  object 
 10  ID_CONNECTED_NUM         17140 non-null  float64
dtypes: float64(4), int64(2), object(5)
memory usage: 6.4+ MB


In [58]:
d_telecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120439 entries, 0 to 120438
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   SESSION_START_TIME       120439 non-null  object 
 1   ID                       120439 non-null  int64  
 2   SERVICE_ID               120439 non-null  int64  
 3   CELL_NUM                 91729 non-null   object 
 4   LAC_NUM                  91739 non-null   object 
 5   TARIFF_PLAN_NAME         120439 non-null  object 
 6   PAY_AMT                  120439 non-null  float64
 7   SESSION_DURATION_IN_SEC  107249 non-null  float64
 8   DIRECT_IND               61161 non-null   float64
 9   TYPE_OF_CONNECTION       107249 non-null  object 
 10  ID_CONNECTED_NUM         56205 non-null   float64
dtypes: float64(4), int64(2), object(5)
memory usage: 10.1+ MB


In [59]:
abon_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31283 entries, 0 to 31282
Data columns (total 71 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           31283 non-null  int64  
 1   VARIABLE_0   14638 non-null  float64
 2   VARIABLE_1   14638 non-null  float64
 3   VARIABLE_2   24555 non-null  float64
 4   VARIABLE_3   24555 non-null  float64
 5   VARIABLE_4   31283 non-null  float64
 6   VARIABLE_5   31283 non-null  int64  
 7   VARIABLE_6   31283 non-null  int64  
 8   VARIABLE_7   31283 non-null  int64  
 9   VARIABLE_8   31283 non-null  int64  
 10  VARIABLE_9   31283 non-null  int64  
 11  VARIABLE_10  31283 non-null  float64
 12  VARIABLE_11  31283 non-null  int64  
 13  VARIABLE_12  31283 non-null  int64  
 14  VARIABLE_13  31283 non-null  int64  
 15  VARIABLE_14  31283 non-null  int64  
 16  VARIABLE_15  31283 non-null  int64  
 17  VARIABLE_16  31283 non-null  float64
 18  VARIABLE_17  31283 non-null  int64  
 19  VARI

In [60]:
# Merge the K-Telecom data with the subscriber profile data to identify former clients
k_telecom = pd.merge(k_telecom, abon_data, on="ID", how="inner")
k_telecom.shape
# print(len(k_telecom))


(75993, 81)

In [61]:
d_telecom = pd.merge(d_telecom, abon_data, on="ID", how="inner")
d_telecom.shape

(120242, 81)

In [63]:
k_telecom_subscribers = set(k_telecom["ID"])
d_telecom_subscribers = set(d_telecom["ID"])

print("Length of k_telecom_subscribers set:", len(k_telecom_subscribers))
print("Length of d_telecom_subscribers set:", len(d_telecom_subscribers))


Length of k_telecom_subscribers set: 8781
Length of d_telecom_subscribers set: 22502


In [65]:

# Function to check if a subscriber of D-Telecom was a former client of K-Telecom
def check_former_client(subscriber_id):
    if subscriber_id in k_telecom_subscribers:
        return 1
    else:
        return 0

In [76]:
# New data frame of d_telecom.

new_d_telecom = d_telecom['ID']
new_d_telecom = new_d_telecom.to_frame()
new_d_telecom.drop_duplicates(subset='ID')
print("Shape of D Telecom : ", new_d_telecom.shape)

new_d_telecom["EX_K_TELECOM_IND"] = new_d_telecom["ID"].apply(check_former_client)
new_d_telecom['EX_K_TELECOM_IND'].value_counts()

Shape of D Telecom :  (120242, 1)


EX_K_TELECOM_IND
0    120242
Name: count, dtype: int64

In [68]:
d_telecom.head()

,SESSION_START_TIME,ID,SERVICE_ID,CELL_NUM,LAC_NUM,TARIFF_PLAN_NAME,PAY_AMT,SESSION_DURATION_IN_SEC,DIRECT_IND,TYPE_OF_CONNECTION,...,VARIABLE_61,VARIABLE_62,VARIABLE_63,VARIABLE_64,VARIABLE_65,VARIABLE_66,VARIABLE_67,VARIABLE_68,VARIABLE_69,EX_K_TELECOM_IND
0,2019-05-19 11:27:51.0,73811,1140,CELL_2232,LAC_589,PRICE_PLAN_NAME_87,0.031415,0.0,NaN,gprs,...,0.279296,0.655167,0.339549,465.593708,2092.486501,0.815618,0.261985,0.766966,0.321684,0
1,2019-05-18 09:09:33.0,73811,1141,CELL_2232,LAC_589,PRICE_PLAN_NAME_87,0.031415,900.0,NaN,gprs,...,0.279296,0.655167,0.339549,465.593708,2092.486501,0.815618,0.261985,0.766966,0.321684,0
2,2019-06-23 00:01:44.0,73811,1049,NaN,NaN,PRICE_PLAN_NAME_87,62.830000,0.0,2.0,sms,...,0.279296,0.655167,0.339549,465.593708,2092.486501,0.815618,0.261985,0.766966,0.321684,0
3,2019-05-18 09:49:35.0,73811,1140,CELL_2232,LAC_589,PRICE_PLAN_NAME_87,0.879620,17730.0,NaN,gprs,...,0.279296,0.655167,0.339549,465.593708,2092.486501,0.815618,0.261985,0.766966,0.321684,0
4,2019-05-19 19:02:14.0,10707,1069,CELL_17104,LAC_646,PRICE_PLAN_NAME_105,0.125660,4451.0,NaN,gprs,...,0.000000,0.000000,0.000000,19.916986,75.268106,0.000000,0.000000,0.000000,0.000000,0


In [69]:
# Cross Check

selected_id = k_telecom["ID"].iloc[0]
print("Selected ID :", selected_id)
result = check_former_client(selected_id)
print("Result for subscriber ID", ":", result)


Selected ID : 62340
Result for subscriber ID : 1


In [70]:
d_selected_id = d_telecom["ID"].iloc[0]
print("Selected ID :", d_selected_id)
result = check_former_client(d_selected_id)
print("Result for subscriber ID", ":", result)


Selected ID : 73811
Result for subscriber ID : 0


Based on the data exists there, the K_Telecome has about 8K customer based and D_telecom has around 22K of customer based.

Finally, there is no churning rate of K_telecom went to D_telecom on any reason.